In [1]:
import pandapower as pp
import numpy as np
import cvxpy as cp
from scipy.linalg import block_diag

In [2]:
# extract impedance matrix from loaded network file
def impedance_matrix(filename):
    net = pp.converter.from_mpc(filename, f_hz=60)

    # compute the admittance matrix
    pp.runpp(net, numda=False, max_iteration=10)

    matrix_Y = np.array(net._ppc["internal"]["Ybus"].todense())

    # Based on Bolognani's paper, we need to exclude slack bus to get Z
    matrix_Y = matrix_Y[1:, :]
    matrix_Y = matrix_Y[:, 1:]
    matrix_Z = np.linalg.inv(matrix_Y)

    matrix_XX = np.imag(matrix_Z) * 121
    matrix_RR = np.real(matrix_Z) * 121

    return matrix_RR, matrix_XX

In [3]:
filename = 'case34sa_mod.mat'

matrix_R, matrix_X = impedance_matrix(filename)

/Users/zhangpeng/miniforge3/envs/Master_Winter23/lib/python3.9/site-packages/pandapower/create.py:4901: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  net[table] = pd.concat([net[table], dd], sort=False)
/Users/zhangpeng/miniforge3/envs/Master_Winter23/lib/python3.9/site-packages/pandapower/converter/pypower/from_ppc.py:277: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  branch_lookup["element"].loc[~is_line] = idx_trafo


In [4]:
n = matrix_R.shape[0]  # number of nodes
m = 2 * n

A = np.eye(n)
#A = np.zeros((n,n)) # menoryless system
B = np.hstack([matrix_R, matrix_X])
Q = 10 * np.eye(n) # cost of safety 
R = np.eye(m) # cost of u
QR = block_diag(Q, R)

AB = np.hstack([A, B])

sigma = 100000000 # we just need to make sure it's sufficienly large 100000000

W = np.eye(n) # adjust it based on w


# Define the variable
X = cp.Variable((n+m, n+m), symmetric=True)

# Define the objective
objective = cp.Minimize(0)

# Define the constraints
constraints = [X >> 0]  # X is symmetric positive semidefinite
constraints += [cp.trace(X) <= sigma]
transient = AB @ X @ AB.T
for i in range(n):
    for j in range(n):
        constraints += [X[i, j] == (transient[i, j] + W[i, j])]

# Form the problem
prob = cp.Problem(objective, constraints)

# Solve the problem using MOSEK
prob.solve(solver=cp.MOSEK)

# Print the results
print("The optimal value is:", prob.value)
print("The optimal X is:")
print(X.value)


The optimal value is: 0.0
The optimal X is:
[[ 978773.18226764   33882.18326552   35150.54879008 ...   -4533.94954624
    -4569.9179809    -4583.70978317]
 [  33882.18326552 1018538.3207536    63433.73598621 ...   -8775.21387312
    -8838.39474893   -8863.48653473]
 [  35150.54879008   63433.73598621 1043116.60829245 ...   -8058.63029375
    -8080.37846931   -8093.27594376]
 ...
 [  -4533.94954624   -8775.21387312   -8058.63029375 ...  953800.56819658
    -4670.71166603   -4689.76600084]
 [  -4569.9179809    -8838.39474893   -8080.37846931 ...   -4670.71166603
   953642.35399976   -4777.79954665]
 [  -4583.70978317   -8863.48653473   -8093.27594376 ...   -4689.76600084
    -4777.79954665  953583.720149  ]]


In [5]:
X_value = X.value

X1 = np.ones((m, n))
X2 = np.ones((n,n))

for i in range(m):
    for j in range(n):
        X1[i, j] = X_value[i+n, j]

for i in range(n):
    for j in range(n):
        X2[i, j] = X_value[i, j]

KK = X1 @ np.linalg.inv(X2)

print(KK)

# np.savetxt('Controller_K.csv', KK, delimiter=',')
#np.save('Controller_K.npy', KK)
#validate this KK is stabilzing? or it has been proved in Xinyi's paper

[[-0.04145986 -0.02657125 -0.01690959 ...  0.0008856   0.00074853
   0.00070021]
 [-0.02661579 -0.05218988 -0.03362527 ...  0.00173047  0.00148655
   0.00139845]
 [-0.01633813 -0.03245848 -0.07843498 ...  0.0037935   0.00347421
   0.0033594 ]
 ...
 [-0.00353632 -0.00699559 -0.00602373 ... -0.01156302 -0.0069043
  -0.00533571]
 [-0.00356421 -0.0070461  -0.00608469 ... -0.00680158 -0.01485112
  -0.01298907]
 [-0.00357423 -0.00706498 -0.00610806 ... -0.00519504 -0.01295098
  -0.01995483]]


### Validation

In [6]:
rho = A + B @ KK

# Compute the eigenvalues
eigenvalues = np.linalg.eigvals(rho)

# Compute the spectral radius
spectral_radius = max(eigenvalues)

print(spectral_radius)

0.9997502185671757
